In [4]:
import requests
import gzip
import shutil
import os
import json
from dotenv import load_dotenv

In [3]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv()
FAERS_API_KEY = os.getenv('FAERS_API_KEY')

In [18]:
def download_sider_data():
    url = 'http://sideeffects.embl.de/media/download/meddra_all_label_se.tsv.gz'
    # Obtener la ruta del directorio actual del script
    script_dir = os.getcwd()
    
    # Construir la ruta relativa al directorio 'data/raw'
    local_filename = os.path.join(script_dir, '..', '..', 'data', 'raw', 'meddra_all_label_se.tsv.gz')
        
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    print("Download and extraction complete from SIDER.")

def unzip_sider_data():
    # Obtener la ruta del directorio actual del script
    script_dir = os.getcwd()
    
    # Construir la ruta relativa al directorio 'data/raw'
    local_filename = os.path.join(script_dir, '..', '..', 'data', 'raw', 'meddra_all_label_se.tsv.gz')
    extracted_filename = os.path.join(script_dir, '..', '..', 'data', 'raw', 'sider_side_effects.tsv')
    
    # Verificar si el archivo sider_side_effects.tsv ya existe
    if not os.path.exists(extracted_filename):
        print("The file side_effects.tsv does not exist. Creating file...")
        # Si no existe, crear el archivo
        with open(extracted_filename, 'w'):
            pass  # Esto crea el archivo vacío
        print("File successfully created.")
        
    else:
        print("The file side_effects.tsv already exists.")
    
    print("Unzipping SIDER file...")
    with gzip.open(local_filename, 'rb') as f_in:
        with open(extracted_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            print("Unzipped SIDER file")

def download_openfda_data():
    base_url = 'https://api.fda.gov/drug/event.json'
    params = {
        'api_key': FAERS_API_KEY,
        'limit': 100  # Ajusta el límite según sea necesario
    }
    headers = {
        'Content-Type': 'application/json',
    }
    
    # Obtener la ruta del directorio actual del script
    notebook_dir = os.getcwd()

    # Construir la ruta relativa al directorio 'data/raw'
    local_filename = os.path.join(notebook_dir, '..', '..', 'data', 'raw', 'openfda_data.json')
    
    try:
        
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()
        
        # Convertir la respuesta a JSON
        data = response.json()
             
        with open(local_filename, 'w') as f:
            json.dump(data, f)
        print("Download of OpenFDA data complete.")
        return data
     
    except requests.exceptions.RequestException as e:
        print(f"Error en la llamada a la API: {e}")
        
        # Si hay una excepción HTTPError, imprimir la información del error
        if isinstance(e, requests.exceptions.HTTPError):
            print(f"Status Code: {e.response.status_code}")
            print(f"Response: {e.response.text}")


In [19]:
df = download_openfda_data()
df

Download of OpenFDA data complete.


{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
  'terms': 'https://open.fda.gov/terms/',
  'license': 'https://open.fda.gov/license/',
  'last_updated': '2024-04-23',
  'results': {'skip': 0, 'limit': 100, 'total': 17707126}},
 'results': [{'safetyreportid': '5801206-7',
   'transmissiondateformat': '102',
   'transmissiondate': '20090109',
   'serious': '1',
   'seriousnessdeath': '1',
   'receivedateformat': '102',
   'receivedate': '20080707',
   'receiptdateformat': '102',
   'receiptdate': '20080625',
   'fulfillexpeditecriteria': '1',
   'companynumb': 'JACAN16471',
   'primarysource': {'reportercountry': 'CANADA', 'qualification': '3'},
   'sender': {'senderorganization': 'FDA-Public Use'},
   'receiver': None,
   'patient': {'patientonse

In [20]:
download_sider_data()

Download and extraction complete from SIDER.


In [ ]:
unzip_sider_data()